In [2]:
pip install torch torchvision matplotlib

                                              0.0/212.5 MB ? eta -:--:--
                                             1.8/212.5 MB 56.5 MB/s eta 0:00:04
                                             5.1/212.5 MB 65.4 MB/s eta 0:00:04
     -                                       7.8/212.5 MB 62.7 MB/s eta 0:00:04
     --                                     11.2/212.5 MB 65.2 MB/s eta 0:00:04
     --                                     15.7/212.5 MB 81.8 MB/s eta 0:00:03
     ---                                   20.7/212.5 MB 110.0 MB/s eta 0:00:02
     ----                                  26.3/212.5 MB 131.2 MB/s eta 0:00:02
     -----                                 32.7/212.5 MB 131.2 MB/s eta 0:00:02
     ------                                 34.5/212.5 MB 93.9 MB/s eta 0:00:02
     ------                                 36.2/212.5 MB 72.6 MB/s eta 0:00:03
     ------                                 36.9/212.5 MB 65.6 MB/s eta 0:00:03
     ------                                 38.


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import sys
import os

# Ruta absoluta al proyecto (ajústala si trabajas en otro entorno)
PROJECT_ROOT = os.path.abspath(os.path.join(".."))  # asumiendo que test1.ipynb está en CW/tests

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [38]:
from models import base_models
print(dir(base_models))

['ABC', 'DifferentiableModel', 'Model', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_preprocessing_fn', 'abc', 'absolute_import', 'abstractmethod', 'np', 'sys']


In [39]:
# Desde ataques
from attacks import base
from attacks import carlini_wagner
from models import base

# Desde el resto de archivos
import criteria
import distances
import utils

# Desde models y tests si necesitas
from models import pytorch

In [42]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from attacks.carlini_wagner import CarliniWagnerL2Attack
from adversarial import Adversarial
from criteria import Misclassification
import distances


In [43]:
model = torchvision.models.resnet18(pretrained=True)
model.eval()

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [49]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Dataset solo para una imagen de ejemplo
imagenet_val = torchvision.datasets.ImageNet(root='./imagenet', split='val', transform=transform)
image, label = imagenet_val[0]

# Añadir batch dimension
image = image.unsqueeze(0)
label_tensor = torch.tensor([label])

print(f"Etiqueta real: {label}")


RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in ./imagenet.

In [ ]:
with torch.no_grad():
    output = model(image)
    pred_label = output.argmax(dim=1).item()

print(f"Predicción original: {pred_label}")


In [ ]:
# Crear objeto Adversarial
criterion = Misclassification()
adv = Adversarial(model=model,
                  criterion=criterion,
                  original_image=image,
                  original_class=label,
                  distance=l2)

# Ejecutar ataque
attack = CarliniWagnerL2Attack()
adversarial_example = attack(adv)

# Verificar
if adv.is_adversarial:
    print(f"¡Ejemplo adversarial generado! Nueva predicción: {adv.adversarial_class}")
else:
    print("No se generó adversarial.")


In [ ]:
def show_image(tensor, title=""):
    img = tensor.squeeze().detach().numpy().transpose(1, 2, 0)
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')

plt.figure(figsize=(10, 3))

plt.subplot(1, 3, 1)
show_image(image, "Original")

plt.subplot(1, 3, 2)
show_image(adv.adversarial_image, "Adversarial")

plt.subplot(1, 3, 3)
show_image((adv.adversarial_image - image).abs(), "Perturbación")

plt.show()


In [ ]:
print("Distancia L2:", adv.distance.value)


In [ ]:

# Mostrar imágenes
def show(img1, img2, title1="Original", title2="Adversarial"):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].imshow(img1)
    axs[0].set_title(title1)
    axs[0].axis("off")

    axs[1].imshow(img2)
    axs[1].set_title(title2)
    axs[1].axis("off")

    plt.tight_layout()
    plt.show()

if adversarial is not None:
    adv_img_np = adversarial.squeeze().permute(1, 2, 0).numpy()
    show(img_np, adv_img_np)

    # Predicción adversarial
    adv_logits = model(adversarial)
    adv_label = torch.argmax(adv_logits, dim=1).item()
    print(f"Etiqueta adversarial predicha: {adv_label}")
else:
    print("No se generó una imagen adversarial.")
